In [9]:
import  numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import scipy.ndimage

In [10]:
def preprocess_img(img,thresh=1):
    new_img = cv2.GaussianBlur(img,(3,3),0)
    __,new_img = cv2.threshold(new_img,170,255,cv2.THRESH_BINARY_INV)
    kernel = np.ones((3,3),np.uint8)
    new_img = cv2.dilate(new_img,kernel,iterations = 2)
    new_img = cv2.erode(new_img,kernel,iterations=1)
    return new_img
def getMeanArea(contours):
    meanArea=0
    for contour in contours: meanArea+=cv2.contourArea(contour)
    meanArea=(meanArea)/len(contours)
    return meanArea  
def getRatioArea(contours):
    meanArea=0
    for contour in contours: meanArea+=cv2.contourArea(contour)
    cnsSorted = sorted(contours, key=lambda x:cv2.contourArea(x), reverse = True)
    ratioArea = cv2.contourArea(cnsSorted[0])/meanArea
    return ratioArea
def purify(img):
    img=cv2.copyMakeBorder(img,32,32,32,32,cv2.BORDER_CONSTANT)
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    meanArea=getMeanArea(contours)
    nlabels,labels,stats,centroids=cv2.connectedComponentsWithStats(img,None,None,None,8,cv2.CV_32S)
    areas=stats[1:,cv2.CC_STAT_AREA]
    result=np.zeros((labels.shape),np.uint8)
    for i in range(nlabels-1):
        if areas[i]>=0.1*meanArea:
            result[labels==i+1]=255
    high=max(result.shape[0],result.shape[1])
    if high==result.shape[0]:
        dif=(high-result.shape[1])//2
        result=cv2.copyMakeBorder(result,0,0,dif,dif,cv2.BORDER_CONSTANT,value=0)
    else:
        dif=(high-result.shape[1])//2
        result=cv2.copyMakeBorder(result,dif,dif,0,0,cv2.BORDER_CONSTANT,value=0)
    return cv2.resize(result,(28,28),interpolation=cv2.INTER_AREA)
def extract_character(image, recursion = 0):
    thresh = cv2.copyMakeBorder(image, 8, 8, 8, 8, cv2.BORDER_REPLICATE)
    pad=5
    thresh=cv2.GaussianBlur(thresh, (3,3), 0)
    ret,thresh=cv2.threshold(thresh,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    kernel1 = np.ones((3,3), np.uint8)
    thresh = cv2.dilate(thresh, kernel1, iterations = 1)    
    if(recursion<2):
    	thresh2 = cv2.erode(thresh, np.ones((2,2), np.uint8), iterations = 2)
    	thresh2 = scipy.ndimage.median_filter(thresh2, (5, 1)) # remove line noise
    	thresh2 = scipy.ndimage.median_filter(thresh2, (1, 5)) # weaken circle noise
    	thresh2 = scipy.ndimage.median_filter(thresh2, (5, 1)) # remove line noise
    	thresh2 = scipy.ndimage.median_filter(thresh2, (1, 5)) # weaken circle noise
    	contours1, hierarchy = cv2.findContours(thresh2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    else: contours1, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    coords=[]
    count=0
    ratioArea = getRatioArea(contours1)
    if(ratioArea<0.3 or recursion>1): kernel2 = np.ones((2,2), np.uint8)
    elif(ratioArea>0.85 and recursion<1): kernel2 = np.ones((5,5), np.uint8)
    else: kernel2 = np.ones((3,3), np.uint8)
    if(ratioArea > 0.3 and recursion<2):
    	thresh = cv2.erode(thresh, kernel2, iterations = 2)
    	thresh = scipy.ndimage.median_filter(thresh, (5, 1)) # remove line noise
    	thresh = scipy.ndimage.median_filter(thresh, (1, 5)) # weaken circle noise
    	thresh = scipy.ndimage.median_filter(thresh, (5, 1)) # remove line noise
    	thresh = scipy.ndimage.median_filter(thresh, (1, 5)) # weaken circle noise
    thresh = cv2.dilate(thresh, kernel1, iterations = 1)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    coords=[]
    count=0
    meanArea=getMeanArea(contours)
    for contour in contours:
        (x,y,w,h)=cv2.boundingRect(contour)
        if cv2.contourArea(contour)>0.05*meanArea:
            if w / h > 1.25:  #Split it in half into two letter regions
                half_width = int(w / 2)
                coords.append((x, y, half_width, h))
                coords.append((x + half_width, y, half_width, h))
                count=count+2
            else:  
                coords.append((x, y, w, h))
                count=count+1
    coords=sorted(coords,key=lambda x: x[0])
    img_paths=[]
    if(count >7 and recursion <3):
    	img_paths_array = extract_character(image, recursion + 1)
    	return img_paths_array
    else:
    	for i in range(count):
        	result=purify(thresh[coords[i][1]:coords[i][1]+coords[i][3],coords[i][0]:coords[i][0]+coords[i][2]])
        	filename='character'+str(i)+'.jpg'
        	cv2.imwrite(filename,cv2.bitwise_not(result))
        	img_paths.append(filename)
    	return np.array(img_paths)
def crop_captcha(test):
    u = np.where((np.sum(test,axis=1)/np.sum(test))!=0)[0].min()
    d = np.where((np.sum(test,axis=1)/np.sum(test))!=0)[0].max()
    l = np.where((np.sum(test, axis=0)/np.sum(test))!=0)[0].min()
    r = np.where((np.sum(test, axis=0)/np.sum(test))!=0)[0].max()
    return test[u:d,l:r]
def segment(img, thresh1=0.001, thresh2=1.2):
  test = img.copy()
  arr = np.where((np.sum(test, axis=0)/np.sum(test)) < thresh1/100)[0]
  i = 0
  l,r = list(), list()
  while(i<len(arr)):
    l.append(arr[i])
    i+=1
    while(i<(len(arr)-1) and (arr[i+1]==arr[i]+1 or arr[i+1]==arr[i])):
      i+=1
    r.append(arr[i])
    i+=1
  ret = list()
  prev = 0
  for i in range(len(l)):
    x = (l[i]+r[i])//2
    
    if (x!=prev): 
      ret.append(test[:, prev:x])
    # plt.imshow(ret)
    prev = x
  ret.append(test[:, prev:])
  return ret
def add_border(img, shape):
  if img.shape[0] > img.shape[1]:
    x = (img.shape[0]-img.shape[1])
    y = x//2
    img = cv2.copyMakeBorder(
        img,
        top = 10,
        bottom = 10,
        left = y+10,
        right = x-y+10,
        borderType = cv2.BORDER_CONSTANT,
        value = 0 
    )
  else:
    x = (img.shape[1]-img.shape[0])
    y = x//2
    img = cv2.copyMakeBorder(
        img,
        top = y+10,
        bottom = x-y+10,
        left = 10,
        right = 10,
        borderType = cv2.BORDER_CONSTANT,
        value = 0 
    )
  img = cv2.resize(img, (shape,shape), interpolation=cv2.INTER_CUBIC)#
  return img
def centre_img(test):
  test = cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
  # plt.imshow(test)
  M = cv2.moments(test, binaryImage=True)
  if M["m00"] != 0:
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
  else:
    cX, cY = 0, 0
  height, width = test.shape[:2]
  translation_matrix = np.array([
    [1, 0, width//2-cX],
    [0, 1, height//2-cY],
  ], dtype=np.float32)
  translated_image = cv2.warpAffine(src=test, M = translation_matrix, dsize=(width, height))
  return translated_image

In [11]:
emodel = tf.keras.models.load_model('jaishreeram_emoji.h5')
lmodel = tf.keras.models.load_model('jaishreeram_letters.h5',compile=True)

In [12]:
def letter_solver(path):
    classes=' ABCDEFGHIJKLMNOPQRSTUVWXYZ01234567891234567'
    image = cv2.imread(path)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

    img_paths=extract_character(image)
    output=' '
    for i in img_paths:
        m=[]
        img=cv2.imread(i,cv2.IMREAD_GRAYSCALE)
        img = cv2.bitwise_not(img)
        # cv2.imshow('img',img)
        # cv2.waitKey(0)
        img=np.reshape(img,(28,28,1))/255
        m.append(img)
        m=np.array(m)
        result=np.argmax(lmodel.predict(m))
        output+=classes[result]

    print(output)


In [13]:
def emoji_solver(path):
    ans = ""
    image = cv2.imread(path)
    image = preprocess_img(image)
    image = crop_captcha(image)
    image_list = list()
    for imgs in segment(image,thresh2=3):
        image_list.append(imgs)
    
    for test in image_list:
        testEmoji = add_border(test,300)
        # plt.imshow(test)
        testEmoji = centre_img(testEmoji)
        # plt.imshow(test)
        testEmoji = np.expand_dims(testEmoji.astype(np.float32)/255, [-1,0])
        emojiArray = emodel.predict(testEmoji)
        emojiArray = emojiArray.reshape(emojiArray.shape[1])
        ans  = ans + str(np.argmax(emojiArray)+1)
        # cv2.imshow('img',testEmoji.reshape(300,300))
        # cv2.waitKey(0)
        
    print(ans)

In [14]:
#pass file path to check letters
letter_solver('test1.jpg')

 STUVWXYZ


In [15]:
#pass file path to check emojis
emoji_solver('test9.jpg')

112
